In [ ]:
import geopandas as gpd
import pandas as pd

seq = pd.read_csv(r'')

def stopseq_to_route(df, routename = 'RouteName', seq = 'Seq', Lat = 'PostitionLat', Lon = 'PositionLon'):
    return route_gdf 